In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
# Read data:
train_headers = ["index", "word", "pos_tag"]
train_df = pd.read_csv("./data/train", sep="\t", header=None)
train_df.columns = train_headers

dev_df = pd.read_csv("./data/dev", sep="\t", header=None)
dev_df.columns = train_headers

test_headers = ["index", "word"]
test_df = pd.read_csv("./data/test", sep="\t", header=None)
test_df.columns = test_headers

In [3]:
counter = {}
for row in train_df.iterrows():
    if row[1]["word"] in counter:
        counter[row[1]["word"]] += 1
    else:
        counter[row[1]["word"]] = 1

In [4]:
threshold = 2 
unknown_word_list = []
for key, value in counter.items():
    if value < threshold:
        unknown_word_list.append(key)
    else:
        continue
counter["< unk >"] = len(unknown_word_list)

In [5]:
counter_sorted = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)}

In [6]:
index=0
with open('vocab.txt', 'w') as f:
    f.write("< unk >")
    f.write("\t")
    f.write(str(index))
    f.write("\t")
    f.write(str(counter["< unk >"]))
    f.write("\n")
    index+=1
    for key, value in counter.items():
        if key == "< unk >":
            continue
        elif value >= threshold:
            f.write(key)
            f.write("\t")
            f.write(str(index))
            f.write("\t")
            f.write(str(value))
            f.write("\n")
            index+=1
        

In [7]:
print("Size of vocabulary:", index)
print("Unknown word count", len(unknown_word_list))

Size of vocabulary: 23183
Unknown word count 20011


In [8]:
vocab = {} #want just 0,word1... 90,word90 mapping
file = open("vocab.txt", "r").read().splitlines()


for line in file:
    line_split = line.split("\n")
    for actual_line in line_split:
        actual_line_split = actual_line.split("\t")
        if len(actual_line_split) == 1:
            break
        vocab[int(actual_line_split[1])] = actual_line_split[0]

In [9]:
pos_tag_dict = {}
pos_tags = train_df["pos_tag"].values
for i, pos_tag in enumerate(pos_tags):
    if i == (len(pos_tags) - 1):
        break
    
    next_tag = pos_tags[i+1]
    if pos_tag in pos_tag_dict:
        if next_tag in pos_tag_dict[pos_tag]:
            pos_tag_dict[pos_tag][next_tag] += 1
        else:
            pos_tag_dict[pos_tag][next_tag] = 1
            
    else:
        pos_tag_dict[pos_tag] = {}
        pos_tag_dict[pos_tag][next_tag] = 1

In [11]:
pos_total_counts_d = {}
for key, value in pos_tag_dict.items():
    pos_total_counts_d[key] = 0
    for k_inner, v_inner in value.items():
        pos_total_counts_d[key] += v_inner

In [14]:
transition_d = {}
for key, value in pos_tag_dict.items():
    transition_d[key] = {}
    for k_inner, v_inner in value.items():
        transition_d[key][k_inner] = v_inner/pos_total_counts_d[key]

transition_d_formatted = {}
for key, value in transition_d.items():
    for k_inner, v_inner in value.items():
        key_str = str(key) + ", " + str(k_inner)
        transition_d_formatted[key_str] = transition_d[key][k_inner]

In [15]:
pos_to_word_d = {}
all_pos_tags = train_df["pos_tag"].values
all_words = train_df["word"].values
for i, pos_tag in enumerate(all_pos_tags):
    word = all_words[i]
    if pos_tag in pos_to_word_d:
        if word in unknown_word_list:
            if "< unk >" in pos_to_word_d[pos_tag]:
                pos_to_word_d[pos_tag]["< unk >"] += 1
            else:
                pos_to_word_d[pos_tag]["< unk >"] = 1
        else:
            if word in pos_to_word_d[pos_tag]:
                pos_to_word_d[pos_tag][word] += 1
            else:
                pos_to_word_d[pos_tag][word] = 1
            
    else:
        pos_to_word_d[pos_tag] = {}
        pos_to_word_d[pos_tag][word] = 1

In [16]:
emission_d = {}
for k, v in pos_to_word_d.items():
    emission_d[k] = {}
    for k_inner, v_inner in v.items():
        emission_d[k][k_inner] = v_inner/pos_total_counts_d[k]
        
emission_d_formatted = {}
for k, v in emission_d.items():
    for k_inner, v_inner in v.items():
        key_str = str(k) + ", " + str(k_inner)
        emission_d_formatted[key_str] = emission_d[k][k_inner]

In [17]:
results = {}
results["transition"] = transition_d_formatted
results["emission"] = emission_d_formatted

In [18]:
with open('hmm.json', 'w') as f:
    json.dump(results, f)

In [19]:
print("Number of Transition Parameters:", len(transition_d_formatted))
print("Number of Emission Parameters:", len(emission_d_formatted))

Number of Transition Parameters: 1378
Number of Emission Parameters: 30304


In [20]:
hmm = open("hmm.json", "r")
e_t_model = json.load(hmm)

In [ ]:
### 3. Greedy Decoding with HMM:


In [21]:
start = {}
pos_tags_start = train_df[train_df["index"] == 1]["pos_tag"]
for pos_tag in pos_tags_start:
    if pos_tag in start:
        start[pos_tag] += 1
    else:
        start[pos_tag] = 1

In [22]:
start_prob = {k:v/len(pos_tags_start) for k,v in start.items()}

In [28]:
def greedy_hmm(df):
    """Allows for a greedy implementation of HMM. Input - Dataframe that has words as column."""
    pred_lst = []
    all_words = df["word"].values
    prev_pos = "None"
    for i, word in enumerate(all_words):
        best_prob = 0
        best_pos = "None"
        if (word in unknown_word_list) or (word not in counter_sorted):
            word = "< unk >"
        if i == 0:
            for pos_tag in emission_d.keys():
                if word in emission_d[pos_tag]: 
                    e_x1_s1 = emission_d[pos_tag][word]*start_prob[pos_tag]
                    if e_x1_s1 > best_prob:
                        best_prob = e_x1_s1
                        best_pos = pos_tag
                else:
                    continue
            pred_lst.append(best_pos)
            prev_pos = best_pos
        else:
            for pos_tag in emission_d.keys():
                if word in emission_d[pos_tag]:
                    e_x2_s2 = emission_d[pos_tag][word]
                    if pos_tag in transition_d[prev_pos]:
                        t_s2_s1 = transition_d[prev_pos][pos_tag]
                    else:
                        t_s2_s1 = 0
                    prob_ = e_x2_s2*t_s2_s1
                    if prob_ > best_prob:
                        best_prob = prob_
                        best_pos = pos_tag
                else:
                    continue

            if best_pos == "None": #Edge case where word and POS DO NOT Allign
                for pos_tag in emission_d.keys():
                    if word in emission_d[pos_tag]:
                        e_x2_s2 = emission_d[pos_tag][word]
                        if e_x2_s2 > best_prob:
                            best_prob = e_x2_s2
                            best_pos = pos_tag
            pred_lst.append(best_pos)
            prev_pos = best_pos
            
    return pred_lst

In [40]:
train_prediction = greedy_hmm(train_df)

In [44]:
training_accuracy = sum(np.array(train_prediction) == train_df["pos_tag"].values)/len(np.array(train_prediction))
training_accuracy

0.9491895032863902

In [29]:
dev_prediction = greedy_hmm(dev_df)

In [30]:
dev_accuracy = sum(np.array(dev_prediction) == dev_df["pos_tag"].values)/len(np.array(dev_prediction))
dev_accuracy

0.9352877785198227

In [31]:
test_prediction = greedy_hmm(test_df)

In [32]:
with open("greedy.out", "w") as g:
    test_idx = test_df["index"].values
    test_word = test_df["word"].values
    for i, pred in enumerate(test_prediction):
        g.write(str(test_idx[i]))
        g.write("\t")
        g.write(str(test_word[i]))
        g.write("\t")
        g.write(str(pred))
        g.write("\n")

In [ ]:
## 4. Viterbi Decoding With HMM:

In [33]:
def viterbi_hmm(df):
    """Allows for a viterbi decoding implementation of HMM. Input - Dataframe that has words as column."""
    all_words = df["word"].values
    states = [k for k in emission_d.keys()]
    V = []
    for i, word in enumerate(all_words):
        V.append({})
        current_map = {}
        if (word in unknown_word_list) or (word not in counter_sorted):
            word = "< unk >"
        if i == 0:
            for pos_tag in states:
                if word in emission_d[pos_tag]: 
                    e_x1_s1 = np.log(emission_d[pos_tag][word]) + np.log(start_prob[pos_tag])
                    current_map[pos_tag] = {"prob": e_x1_s1, "prev_state": None}
                else:
                    current_map[pos_tag] = {"prob": -50, "prev_state": None}
            V[i] = current_map
        else:
            for pos_tag in states:
                if pos_tag in transition_d[states[0]]:
                    best_prob = V[i-1][states[0]]["prob"] + np.log(transition_d[states[0]][pos_tag])
                else:
                    best_prob = -1000000
                past_st = states[0]
                for prev_tag in states[1:]:
                    if pos_tag in transition_d[prev_tag]:
                        t_s2_s1 = np.log(transition_d[prev_tag][pos_tag])
                    else:
                        t_s2_s1 = -100
                    tr_cost = V[i-1][prev_tag]["prob"] + t_s2_s1 #log or -100 penalty
                    if tr_cost > best_prob:
                        best_prob = tr_cost
                        past_st = prev_tag
                
                if word in emission_d[pos_tag]:
                    e_x1_s1 = np.log(emission_d[pos_tag][word])
                else:
                    e_x1_s1 = -100
                best_prob = best_prob + e_x1_s1  #log or -100 penalty
                V[i][pos_tag] = {"prob": best_prob, "prev_state": past_st}
                
    pred_lst = []
    final_state = "." #init a guess of . ending
    best_prob = -np.inf #init a guess of best prob
    
    #Figure out what the best final state is:
    for st, d_ in V[len(all_words)-1].items():
        if d_["prob"] > best_prob:
            final_state = st
            best_prob = d_["prob"]
            
    #Follow path from end to beginning      
    for i in range(len(all_words)-2, -1, -1):
        pred_lst.append(V[i+1][final_state]["prev_state"])
        final_state = V[i+1][final_state]["prev_state"]
        
    return np.flip(np.array(pred_lst),axis=0)

In [34]:
train_prediction = viterbi_hmm(train_df.iloc[0:100000,:])


In [35]:
train_accuracy = sum(1 for x,y in zip(train_prediction, train_df["pos_tag"].values[0:100000]) if x == y)/len(train_prediction)
train_accuracy

0.9612896128961289

In [36]:
dev_prediction = viterbi_hmm(dev_df)

In [37]:
dev_accuracy = sum(1 for x,y in zip(dev_prediction, dev_df["pos_tag"].values) if x == y)/len(dev_prediction)
dev_accuracy

0.9480294762725113

In [38]:
viterbi_test_prediction = viterbi_hmm(test_df)

In [39]:
with open("viterbi.out", "w") as h:
    test_idx = test_df["index"].values
    test_word = test_df["word"].values
    for i, pred in enumerate(viterbi_test_prediction):
        h.write(str(test_idx[i]))
        h.write("\t")
        h.write(str(test_word[i]))
        h.write("\t")
        h.write(str(pred))
        h.write("\n")